In [1]:
import numpy as np
import sys

sys.path.append('../')

from mandala.nodecore import Node
from mandala.nodecore import Variable

In [2]:
x = Variable(np.ones((3, 3), dtype=np.float32) * 5)

y = x ** 2 + 2 * x + 1

In [4]:
print(y._data)

None


In [5]:
y.data

array([[36., 36., 36.],
       [36., 36., 36.],
       [36., 36., 36.]], dtype=float32)

In [6]:
y[0:1].data

array([[36., 36., 36.]], dtype=float32)

In [7]:
print(y._data)

[[36. 36. 36.]
 [36. 36. 36.]
 [36. 36. 36.]]


## Linear

In [8]:
class Linear(object):
    def __init__(self, in_ch, out_ch):
        self.W = Variable(
            np.random.normal(0, 1 / out_ch, (out_ch, in_ch)).astype('f')
        )

    def forward(self, x):
        self.x = x
        self.y = Node(linear_forward, [self.x, self.W])
        return self.y

    def backward(self):
        self.W.grad += Node(linear_backward_W, [self.x, self.y.grad])
        self.x.grad += Node(linear_backward_x, [self.W, self.y.grad])
    
    def __call__(self, x):
        return self.forward(x)

def linear_forward(x, W):
    y = np.matmul(x, W.T)
    return y

def linear_backward_W(x, gy):
    gW = np.matmul(gy.T, x)
    return gW

def linear_backward_x(W, gy):
    gx = np.matmul(gy, W)
    return gx

In [9]:
l0 = Linear( 5, 10)
l1 = Linear(10, 10)
l2 = Linear(10,  3)

In [10]:
# 真の係数
W = np.random.random((3, 5)).astype(np.float32)

In [11]:
batchsize = 8
x = Variable(np.random.random((batchsize, 5)).astype(np.float32))
t = Variable(np.matmul(x.data, W.T))

In [12]:
h0 = l0(x)
h1 = l1(h0)
y  = l2(h1)

loss = (y - t) ** 2

In [13]:
loss.data

array([[1.5269583 , 1.913333  , 2.4081185 ],
       [1.128682  , 1.6154696 , 1.611495  ],
       [1.0905039 , 1.4177212 , 1.7818569 ],
       [1.618128  , 1.9144117 , 2.0697277 ],
       [1.090003  , 1.4515169 , 1.2977188 ],
       [0.3115629 , 0.4372719 , 0.47729376],
       [0.9048046 , 1.011385  , 1.3853681 ],
       [0.8896132 , 0.74017334, 0.92525643]], dtype=float32)